# ensemble

학습된 하나의 모델을 사용하는 것보다 학습된 여러개의 모델을 사용하면 보다 정확한 예측을 기대할 수 있다.

voting은 여러 예측 모델을 두고 각 모델이 voting하여 최종 결정을 하며,\
bagging은 하나의 모델을 여러개의 데이터 샘플링하여 학습하고 voting하여 최종 결정한다.\
각각 데이터 샘플링하는 방법은 부트스트래핑 분할방식이라고 한다.\
하드보팅은 최종 결정에서 여러개의 모델이 선택한 결정을 따르며, 소프트 보팅은 각 모델이 게인값을 갖고 최종 결정을 한다.

random forest는 bagging의 대표적인 방법이며,\
부트스트래핑으로 샘플링한 데이터마다 예측한 결과를 소프트 보팅하여 최종 예측 결정을 한다.\
Random forest는 앙상블 방법 중 속도가 빠르고 높은 성능을 보여준다.\
Human Activity Recognition Data를 Random forest 기법을 활용하여 분석해본다.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


In [ ]:
url =  'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/features.txt'
feature_name_df = pd.read_csv(url, sep='\s+', header=None, names=['column_index', 'column_name'])
feature_name = feature_name_df.iloc[:,1].values.tolist()

In [ ]:
x_train_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/X_train.txt'
x_test_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/X_test.txt'
y_train_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/y_train.txt'
y_test_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/y_test.txt'

x_train = pd.read_csv(x_train_url, sep='\s+', header=None)
x_test = pd.read_csv(x_test_url, sep='\s+', header=None)
y_train = pd.read_csv(y_train_url, sep='\s+', header=None, names=['action'])
y_test = pd.read_csv(y_test_url, sep='\s+', header=None, names=['action'])

x_train.columns = feature_name
x_test.columns = feature_name

In [ ]:
# 결정나무의 정확도

dt_clf = DecisionTreeClassifier(max_depth=5, random_state=5)
dt_clf.fit(x_train, y_train)
pred = dt_clf.predict(x_test)
accuracy_score(y_test, pred)

In [ ]:
# GridSearch

params = {'max_depth':list(range(6,18,2))}
grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, return_train_score=True)
grid_cv.fit(x_train, y_train)

In [ ]:
# max_depth에 따른 스코어

cv_result_df = pd.DataFrame(grid_cv.cv_results_)
cv_result_df[['param_max_depth', 'mean_test_score', 'mean_train_score']]

In [ ]:
# max_depth가 8에서 가장 점수가 높다는 것을 알았다.
# 최상의 depth인지 과적합의 결과물인지 확인한다.

max_depth = list(range(6,17))

for depth in max_depth :
    dt_clf = DecisionTreeClassifier(max_depth=depth, random_state=5)
    dt_clf.fit(x_train, y_train)
    pred = dt_clf.predict(x_test)
    accuracy = accuracy_score(y_test, pred)
    print('Max_Depth = ', depth, ', Accuracy = ', accuracy)

In [ ]:
best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(x_test)
accuracy_score(y_test, pred1)

In [ ]:
# random forest로 확인한다.

params = {
    'max_depth' : [6, 8, 10],
    'n_estimators' : [50, 100, 200],
    'min_samples_leaf' : [8, 12],
    'min_samples_split' : [8, 12]
}

rf_clf = RandomForestClassifier(random_state=5, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf, param_grid=params, cv=2, n_jobs=-1)
grid_cv.fit(x_train, y_train)

In [ ]:
cv_results_df = pd.DataFrame(grid_cv.cv_results_)
cv_results_df.columns

In [ ]:
target_col = ['rank_test_score', 'mean_test_score', 'param_n_estimators', 'param_max_depth']
cv_results_df[target_col].sort_values('rank_test_score').head()

In [ ]:
grid_cv.best_params_

In [ ]:
grid_cv.best_score_

In [ ]:
rf_clf_best = grid_cv.best_estimator_
rf_clf_best.fit(x_train, y_train)
pred1 = rf_clf_best.predict(x_test)
accuracy

In [ ]:
best_cols_values = rf_clf_best.feature_importances_
best_cols = pd.Series(best_cols_values, index=x_train.columns)
top20_cols = best_cols.sort_values(ascending=False)[:20]
top20_cols

In [ ]:
plt.figure(figsize=(8,8))
sns.barplot(x=top20_cols, y=top20_cols.index)
plt.show

In [ ]:
x_train_re = x_train[top20_cols.index]
x_test_re = x_test[top20_cols.index]
rf_clf_best_re = grid_cv.best_estimator_
rf_clf_best_re.fit(x_train_re, y_train.values.reshape(-1,))
pred1_re = rf_clf_best_re.predict(x_test_re)
accuracy_score(y_test, pred1_re)